# Retrieval-Augmented Generation (RAG) with open-source Hugging Face LLMs using LangChain

![RAG pic](pictures/RAG.png)

## Introduction: 

**Retrieval-Augmented Generation (RAG)** is an approach in natural language processing (NLP) that enhances the capabilities of generative models by integrating external knowledge retrieval into the generation process. This technique aims to improve the quality, relevance, and factual accuracy of the generated text by allowing the model to dynamically access and incorporate information from a large corpus of documents or databases during the generation task. The process involves two key components: a retrieval system and a generative model.

**Working Mechanism**
The working mechanism of RAG typically involves the following steps:

- Query Formation: The system formulates a query based on the initial input or prompt. This query is designed to retrieve information that is likely to be relevant to generating the desired output.

- Information Retrieval: The formulated query is used to fetch relevant information from an external database or knowledge base. The retrieval system may return one or more documents, passages, or data entries that match the query.

- Content Integration: The retrieved information, along with the original input, is provided to the generative model. The model then integrates this information to produce a coherent and contextually enriched output.

- Generation: The generative model synthesizes the final text, taking into account both the input and the retrieved external information. This step ensures that the output is not only relevant and informative but also maintains a natural and fluent language style.

Let's get started!

## Library installation
- Create a virtual environment and install the necessary python libraries
- `pip install transformers sentence-tranformers langchain torch faiss-cpu numpy`

## Library configuration

In [1]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_core.vectorstores import VectorStoreRetriever
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline
from langchain import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import os
from urllib.request import urlretrieve
import numpy as np

/Users/anjiang/anaconda3/envs/chatbot/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Document preparation
**We are going to download 4 publications from United States Census Bureau on the following topics:**
- Occupation, Earnings, and Job Characteristics: July 2022
- Household Income in States andMetropolitan Areas: 2022
- Poverty in States and Metropolitan Areas: 2022
- Health Insurance Coverage Status and Type by Geography: 2021 and 2022

We prepare this documents for the LLM to use as a knowledge base.

In [2]:
# Download documents from U.S. Census Bureau to local directory.
os.makedirs("us_census", exist_ok=True)
files = [
    "https://www.census.gov/content/dam/Census/library/publications/2022/demo/p70-178.pdf",
    "https://www.census.gov/content/dam/Census/library/publications/2023/acs/acsbr-017.pdf",
    "https://www.census.gov/content/dam/Census/library/publications/2023/acs/acsbr-016.pdf",
    "https://www.census.gov/content/dam/Census/library/publications/2023/acs/acsbr-015.pdf",
]
for url in files:
    file_path = os.path.join("us_census", url.rpartition("/")[2])
    urlretrieve(url, file_path)

**Split documents to smaller chunks** 

Documents should be: 
- large enough to contain enough information to answer a question, and 
- small enough to fit into the LLM prompt: mistralai/Mistral-7B-v0.1 input tokens limited to 4096 tokens
- small enough to fit into the embeddings model: WhereIsAI/UAE-Large-V1: input tokens limited to 512 tokens (roughly 2,000 characters. Note: 1 token ~ 4 characters).

For this project, we are going to split documents to chunks of roughly 1000 characters with an overlap of 100 characters.

In [3]:
# Load pdf files in the local directory
loader = PyPDFDirectoryLoader("./us_census/")

docs_before_split = loader.load()
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap  = 100,
)
docs_after_split = text_splitter.split_documents(docs_before_split)

In [4]:
docs_after_split[0]

Document(page_content='Health Insurance Coverage Status and Type \nby Geography: 2021 and 2022\nAmerican Community Survey Briefs\nACSBR-015Issued September 2023Douglas Conway and Breauna Branch\nINTRODUCTION\nDemographic shifts as well as economic and govern-\nment policy changes can affect people’s access to health coverage. For example, between 2021 and 2022, the labor market continued to improve, which may have affected private coverage in the United States \nduring that time.\n1 Public policy changes included \nthe renewal of the Public Health Emergency, which \nallowed Medicaid enrollees to remain covered under the Continuous Enrollment Provision.\n2 The American \nRescue Plan (ARP) enhanced Marketplace premium subsidies for those with incomes above 400 percent of the poverty level as well as for unemployed people.\n3', metadata={'source': 'us_census/acsbr-015.pdf', 'page': 0})

In [5]:
avg_doc_length = lambda docs: sum([len(doc.page_content) for doc in docs])//len(docs)
avg_char_before_split = avg_doc_length(docs_before_split)
avg_char_after_split = avg_doc_length(docs_after_split)

print(f'Before split, there were {len(docs_before_split)} documents loaded, with average characters equal to {avg_char_before_split}.')
print(f'After split, there were {len(docs_after_split)} documents (chunks), with average characters equal to {avg_char_after_split} (average chunk length).')

Before split, there were 63 documents loaded, with average characters equal to 3830.
After split, there were 296 documents (chunks), with average characters equal to 864 (average chunk length).


## Text Embeddings with Hugging Face models
At the time of writing, there are 213 text embeddings models for English on the [Massive Text Embedding Benchmark (MTEB) leaderboard](https://huggingface.co/spaces/mteb/leaderboard). For our project, we are picking a lightweight and powerful open-source model called **UAE-Large-V1**, which is placed in the 6th position on the leaderboard with a much less model size compared to the top 5. An alternative option would be **sentence-transformers (S-BERT)** a classical lightweight embedding model which is also widely used. Note that sentence-tranformers embedding dimension is 384, whereas UAE-Large-V1 has 1024 dimensions. 

In [6]:
huggingface_embeddings = HuggingFaceEmbeddings(
    model_name="WhereIsAI/UAE-Large-V1",  # alternatively use "sentence-transformers/all-MiniLM-l6-v2" for a more light-weight experience.
    model_kwargs={'device':'cpu'}, 
    encode_kwargs={'normalize_embeddings': False}
)

No sentence-transformers model found with name WhereIsAI/UAE-Large-V1. Creating a new one with MEAN pooling.


Now we can see how a sample embedding would look like for one of those chunks.

In [7]:
sample_embedding = np.array(huggingface_embeddings.embed_query(docs_after_split[0].page_content))
print("Sample embedding of a document chunk: ", sample_embedding)
print("Size of the embedding: ", sample_embedding.shape)

Sample embedding of a document chunk:  [ 0.04481124  0.28892794  0.01964425 ... -0.70755053 -0.06768651
 -0.41787085]
Size of the embedding:  (1024,)


## Retrieval System for vector embeddings
Once we have a embedding model, we are ready to vectorize all our documents and store them in a vector store to construct a retrieval system. With specifically designed searching algorithms, a retrieval system can do similarity searching efficiently to retrieve relevant documents.

FAISS (Facebook AI Similarity Search) is a library that allows developers to quickly search for embeddings of multimedia documents that are similar to each other. It solves limitations of traditional query search engines that are optimized for hash-based searches, and provides more scalable similarity search functions (nearest-neighbor search implementations).

In [8]:
vectorstore = FAISS.from_documents(docs_after_split, huggingface_embeddings)

In [9]:
query = """What were the trends in median household income across different states in the United States between 2021 and 2022."""  # Sample question, change to other questions you are interested in.
relevant_documents = vectorstore.similarity_search(query)
print(f'There are {len(relevant_documents)} documents retrieved which are relevant to the query. Display the first one:\n')
print(relevant_documents[0].page_content)

There are 4 documents retrieved which are relevant to the query. Display the first one:

for Arizona, Oregon, and Vermont were not statistically different from 
the U.S. median.
From 2021 to 2022, five states—
Alabama, Alaska, Delaware, Florida, 
and Utah—showed a statistically 
significant increase in real median 
household income; 17 states 
showed a decrease. Real median 
household income in 2022 was not 
statistically different from that in 
2021 for 28 states, the District of 
Columbia, and Puerto Rico  
(Table 1).
Median Household Income in the Past 12 Months
for the United States and Puerto Rico: 2022
DCDE
TXCAMT
AZID
NV
NMCOILOR
UT
KSWY
IA
NESDMN
FLND
OKWI
MOWA
AL GA
LAARMI
INPANY
NC
MSTNVA
KYOH
SCME
WVVT
NH
NJMA
CT
MDRI
U.S. median household
income is $74,755.$85,000 or more
$70,000 to $84,999
$60,000 to $69,999
Less than $60,000Income by state
(2022 inﬂation-adjusted
dollars)
U.S. median household income
does not include Puerto Rico.
HIAK
Note: A state abbreviation surrounded

### Create a retriever interface using vectorstore, we'll use it later to construct Q & A chain using LangChain.

In [10]:
# Use similarity searching algorithm and return 3 most relevant documents.
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})

**Now we have our vector store and retrieval system ready. We then need a large language model (LLM) to process information and answer the question.**

## Open-source LLMs from Hugging Face
**There two ways to utilize Hugging Face LLMs: online and local.**

### Hugging Face Hub
The Hugging Face Hub is an platform with over 350k models, 75k datasets, and 150k demo apps (Spaces), all open source and publicly available, in an online platform where people can easily collaborate and build ML together. 

- To use, we should have the huggingface_hub python package installed.
- Set an environment variable called HUGGINGFACEHUB_API_TOKEN with your Hugging Face access token in it.
- Currently, HuggingFace LangChain integration doesn't support the question-answering task, so we can't select HuggingFace QA models for this project. Instead, we select LLMs from the text-generation task category.  

In [11]:
# from langchain_community.llms import HuggingFaceHub

# hf = HuggingFaceHub(
#     repo_id="EleutherAI/gpt-neo-2.7B",
#     model_kwargs={"temperature":0.1, "max_length":500})

#query = """What were the trends in median household income across different states in the United States between 2021 and 2022."""  # Sample question, change to other questions you are interested in.
# hf.invoke(query)

### Hugging Face Local Pipelines

Hugging Face models can be run locally through the HuggingFacePipeline class.

- We need to install transformers python package.
- The Mistral-7B-v0.1 Large Language Model (LLM) is a pretrained generative text model with 7 billion parameters. Mistral-7B-v0.1 outperforms Llama-2-13B on all benchmarks tested. Read the [paper](https://arxiv.org/abs/2310.06825).
- Mistral-7B-v0.1's model size is 3.5GB, while Llama-2–13B has 13 billion parameters and 25GB model size.
- In order to use Llama2, you need to request access from Meta. Mistral-7B-v0.1 is publically available already.

In [12]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

hf = HuggingFacePipeline.from_model_id(
    model_id="mistralai/Mistral-7B-v0.1",
    task="text-generation",
    pipeline_kwargs={"max_new_tokens": 300}
)

llm = hf

Loading checkpoint shards: 100%|██████████████████| 2/2 [00:18<00:00,  9.31s/it]


In [13]:
llm.invoke(query)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'\n\n## Answer (1)\n\nThe data is available here.\n\nThe data is in the form of a table, so you can\'t use `ggplot2` directly. You can use `tidyverse` to convert the table to a data frame and then plot it.\n\n```\nlibrary(tidyverse)\n\n# Read the data\ndata <- read_csv("https://www.census.gov/data/tables/time-series/demo/income-poverty/historical-income-households/cps-historical-income-households.csv")\n\n# Convert the table to a data frame\ndata <- data %>%\n  as_tibble() %>%\n  select(Year, State, Median_Household_Income) %>%\n  mutate(Year = as.numeric(Year))\n\n# Plot the data\nggplot(data, aes(x = Year, y = Median_Household_Income, color = State)) +\n  geom_line() +\n  labs(x = "Year", y = "Median Household Income")\n```\n\nThis will produce the following plot:\n\nComment: Thank you so much! I\'m still learning R and I\'m not sure how to use the data frame.'

**At a glance, our LLM generates some output that might seem plausible but not accurate or factual. That is because it has not been trained on the census data of recent years.**

- OpenAI GPT-3.5 model (for test purpose only)

In [14]:
# from langchain_openai import ChatOpenAI
# chat = ChatOpenAI(temperature=0)
# chat.invoke(query)
# llm = chat

## Q & A chain 
Now we have both the retrieval system for relevant documents and LLM as QA chatbot ready.

We will take our initial query, together with the relevant documents retreived based on the results of our similarity search, to create a prompt to feed into the LLM. The LLM will take the initial query as the question and relevant documents as the context information to generate a result.

Luckily, **LangChain** provides an abstraction of the whole pipeline - **RetrievalQA**

**Let's first construct a proper prompt for our task.**

Prompt engineering is another crucial factor in LLM's performance.

In [15]:
prompt_template = """Use the following pieces of context to answer the question at the end. Please follow the following rules:
1. If you don't know the answer, don't try to make up an answer. Just say "I can't find the final answer but you may want to check the following links".
2. If you find the answer, write the answer in a concise way with five sentences maximum.

{context}

Question: {question}

Helpful Answer:
"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

Call LangChain's RetrievalQA with the prompt above. 

In [16]:
retrievalQA = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)

## Use RetrievalQA invoke method to execute the chain
Note that Input of [invoke method](https://api.python.langchain.com/en/latest/chains/langchain.chains.retrieval_qa.base.RetrievalQA.html#langchain.chains.retrieval_qa.base.RetrievalQA.invoke) needs to be a dictionary.

In [17]:
# Call the QA chain with our query.
result = retrievalQA.invoke({"query": query})

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [18]:
print(result['result'])


The U.S. median household income in 2022 was $74,755, according to the American Community Survey (ACS). The ACS is a nationwide survey that provides data on the social, economic, housing, and demographic characteristics of the U.S. population. The ACS is conducted by the U.S. Census Bureau and is based on a sample of households.

The ACS data show that median household income in the United States increased from 2021 to 2022. The increase was driven by increases in median household income in 20 states, including California, Florida, and Texas. The median household income in the remaining 30 states and the District of Columbia was not statistically different from the U.S. median.

The ACS data also show that median household income in the United States increased from 2021 to 2022. The increase was driven by increases in median household income in 20 states, including California, Florida, and Texas. The median household income in the remaining 30 states and the District of Columbia was n

In [19]:
relevant_docs = result['source_documents']
print(f'There are {len(relevant_docs)} documents retrieved which are relevant to the query.')
print("*" * 100)
for i, doc in enumerate(relevant_docs):
    print(f"Relevant Document #{i+1}:\nSource file: {doc.metadata['source']}, Page: {doc.metadata['page']}\nContent: {doc.page_content}")
    print("-"*100)

There are 3 documents retrieved which are relevant to the query.
****************************************************************************************************
Relevant Document #1:
Source file: us_census/acsbr-017.pdf, Page: 3
Content: for Arizona, Oregon, and Vermont were not statistically different from 
the U.S. median.
From 2021 to 2022, five states—
Alabama, Alaska, Delaware, Florida, 
and Utah—showed a statistically 
significant increase in real median 
household income; 17 states 
showed a decrease. Real median 
household income in 2022 was not 
statistically different from that in 
2021 for 28 states, the District of 
Columbia, and Puerto Rico  
(Table 1).
Median Household Income in the Past 12 Months
for the United States and Puerto Rico: 2022
DCDE
TXCAMT
AZID
NV
NMCOILOR
UT
KSWY
IA
NESDMN
FLND
OKWI
MOWA
AL GA
LAARMI
INPANY
NC
MSTNVA
KYOH
SCME
WVVT
NH
NJMA
CT
MDRI
U.S. median household
income is $74,755.$85,000 or more
$70,000 to $84,999
$60,000 to $69,999
Less than $60

## Conclusion

- Enhanced Accuracy and Relevance: By leveraging external sources, RAG models can generate content that is more accurate, detailed, and relevant to the given context.
- Factuality: It helps in improving the factuality of the generated text, as the information is directly sourced from reliable external databases or knowledge bases.
- Versatility: RAG can be applied to a wide range of NLP tasks, including question answering, text summarization, content creation, and more, enhancing their performance by providing access to a broader range of information.